<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/FineTune_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets  # Instalar paquetes transformers y datasets
!pip install transformers[torch]  # Instalar transformers con soporte para PyTorch
!pip install accelerate  # Instalar accelerate


from datasets import load_dataset  # Importar load_dataset de datasets

dataset = load_dataset("yelp_review_full")  # Cargar conjunto de datos de reseñas de Yelp
dataset["train"][100]  # Imprimir una muestra del conjunto de entrenamiento

"""As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset:"""

from transformers import AutoTokenizer  # Importar AutoTokenizer de transformers

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # Cargar tokenizador BERT

def tokenize_function(examples):  # Definir función para tokenizar
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)  # Aplicar tokenización a todo el conjunto de datos

"""If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:"""

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # Crear subconjunto pequeño de entrenamiento
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))  # Crear subconjunto pequeño de evaluación

"""
Start by loading your model and specify the number of expected labels. From the Yelp Review [dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields), you know there are five labels:
"""

from transformers import AutoModelForSequenceClassification  # Importar AutoModelForSequenceClassification de transformers

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)  # Cargar modelo BERT para clasificación de secuencias con 5 etiquetas

"""
Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:
"""

from transformers import TrainingArguments  # Importar TrainingArguments de transformers

training_args = TrainingArguments(output_dir="test_trainer")  # Definir hiperparámetros de entrenamiento

"""
[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. You'll need to pass [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. The [🤗 Evaluate](https://huggingface.co/docs/evaluate/index) library provides a simple [`accuracy`](https://huggingface.co/spaces/evaluate-metric/accuracy) function you can load with the [evaluate.load](https://huggingface.co/docs/evaluate/main/en/package_reference/loading_methods#evaluate.load) (see this [quicktour](https://huggingface.co/docs/evaluate/a_quick_tour) for more information) function:
"""

!pip install evaluate  # Instalar evaluate
import numpy as np  # Importar numpy
import evaluate  # Importar evaluate

metric = evaluate.load("accuracy")  # Cargar métrica de precisión

"""Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):"""

def compute_metrics(eval_pred):  # Definir función para calcular métricas
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

"""If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:"""

from transformers import TrainingArguments, Trainer  # Importar Trainer de transformers

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")  # Definir hiperparámetros de entrenamiento con evaluación por época

"""Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:"""

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,
)  # Crear objeto Trainer

"""Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):"""

trainer.train()  # Entrenar el modelo





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.615942,0.208000
2,No log,1.097708,0.510000
3,No log,0.974199,0.580000


TrainOutput(global_step=375, training_loss=1.3345125325520832, metrics={'train_runtime': 363.3245, 'train_samples_per_second': 8.257, 'train_steps_per_second': 1.032, 'total_flos': 789354427392000.0, 'train_loss': 1.3345125325520832, 'epoch': 3.0})

In [2]:
# Run text generation pipeline with our next model
from transformers import pipeline  # Importar pipeline de transformers
prompt = "food is very very good, and perfect service"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)  # Crear pipeline de clasificación de texto
result = pipe(f"<s>[INST] {prompt} [/INST] \n\n")  # Realizar inferencia con texto de ejemplo
print(result)  # Imprimir resultado



[{'label': 'LABEL_4', 'score': 0.49614012241363525}]


In [ ]:
"""# Publicar en Hugging Face"""

from huggingface_hub import notebook_login  # Importar notebook_login de huggingface_hub
notebook_login()  # Iniciar sesión en Hugging Face Hub

trainer.push_to_hub()  # Publicar modelo en Hugging Face Hub